## Step 1: Load Libraries

In [112]:
# Install all required packages
!pip install pandas krippendorff statsmodels scikit-learn seaborn matplotlib numpy nltk

  Using cached nltk-3.9.1-py3-none-any.whl.metadata (2.9 kB)
  Using cached click-8.2.1-py3-none-any.whl.metadata (2.5 kB)
  Using cached regex-2024.11.6-cp313-cp313-macosx_11_0_arm64.whl.metadata (40 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
Using cached nltk-3.9.1-py3-none-any.whl (1.5 MB)
Using cached regex-2024.11.6-cp313-cp313-macosx_11_0_arm64.whl (284 kB)
Using cached click-8.2.1-py3-none-any.whl (102 kB)
Using cached tqdm-4.67.1-py3-none-any.whl (78 kB)

[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [138]:
import pandas as pd
import krippendorff
from statsmodels.stats.inter_rater import fleiss_kappa
from sklearn.metrics import cohen_kappa_score
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from statsmodels.stats.inter_rater import aggregate_raters, fleiss_kappa
import itertools
import seaborn as sns
from scipy.stats import entropy
from sklearn.cluster import KMeans
from scipy.special import rel_entr
from scipy.stats import entropy
import itertools
from collections import Counter
import pandas as pd
import ast 
from statsmodels.stats.inter_rater import fleiss_kappa
from itertools import product
from nltk.metrics import agreement
from nltk.metrics import masi_distance

## Step 2: Read and Prepare Data

In [114]:
df = pd.read_csv("./gold_dataset.csv")

In [115]:
df.head()

,HITId,HITTypeId,Title,Description,Keywords,Reward,CreationTime,MaxAssignments,RequesterAnnotation,AssignmentDurationInSeconds,...,LifetimeApprovalRate,Last30DaysApprovalRate,Last7DaysApprovalRate,Input.text-eng,Input.label,Input.parameter,Input.parameter_num_labels,Answer.manifesto_class.labels,Approve,Reject
0,39N6W9XWS05H65P9I0HAUP0E2RZGYY,3BRS3IGD63EV308CIXT46SFWM41TR3,[ACSS] Text labeling Manifesto,Identify the most relevant labels to describe ...,"labeling, text, manifesto",$1.00,Mon Apr 14 05:38:01 PDT 2025,3,BatchId:414859;OriginalHitTemplateId:921587261;,3600,...,100% (152/152),100% (46/46),100% (46/46),"Madam President, Mr Posselt, I do not really t...",0,Par303_Governmental and Administrative Efficiency,2,[],NaN,NaN
1,39N6W9XWS05H65P9I0HAUP0E2RZGYY,3BRS3IGD63EV308CIXT46SFWM41TR3,[ACSS] Text labeling Manifesto,Identify the most relevant labels to describe ...,"labeling, text, manifesto",$1.00,Mon Apr 14 05:38:01 PDT 2025,3,BatchId:414859;OriginalHitTemplateId:921587261;,3600,...,100% (238/239),100% (19/19),100% (19/19),"Madam President, Mr Posselt, I do not really t...",0,Par303_Governmental and Administrative Efficiency,2,[],NaN,NaN
2,39N6W9XWS05H65P9I0HAUP0E2RZGYY,3BRS3IGD63EV308CIXT46SFWM41TR3,[ACSS] Text labeling Manifesto,Identify the most relevant labels to describe ...,"labeling, text, manifesto",$1.00,Mon Apr 14 05:38:01 PDT 2025,3,BatchId:414859;OriginalHitTemplateId:921587261;,3600,...,100% (287/287),100% (75/75),100% (19/19),"Madam President, Mr Posselt, I do not really t...",0,Par303_Governmental and Administrative Efficiency,2,[],NaN,NaN
3,3P7RGTLO71VLC9NPSLO9PVA06HDAKL,3BRS3IGD63EV308CIXT46SFWM41TR3,[ACSS] Text labeling Manifesto,Identify the most relevant labels to describe ...,"labeling, text, manifesto",$1.00,Mon Apr 14 05:38:01 PDT 2025,3,BatchId:414859;OriginalHitTemplateId:921587261;,3600,...,100% (152/152),100% (46/46),100% (46/46),"Mr President, ladies and gentlemen, first of a...",1,Par405_Corporatism,2,"[""Keynesian Demand Management"",""Market regulat...",NaN,NaN
4,3P7RGTLO71VLC9NPSLO9PVA06HDAKL,3BRS3IGD63EV308CIXT46SFWM41TR3,[ACSS] Text labeling Manifesto,Identify the most relevant labels to describe ...,"labeling, text, manifesto",$1.00,Mon Apr 14 05:38:01 PDT 2025,3,BatchId:414859;OriginalHitTemplateId:921587261;,3600,...,100% (287/287),100% (75/75),100% (19/19),"Mr President, ladies and gentlemen, first of a...",1,Par405_Corporatism,2,"[""Government and administrative efficiency"",""M...",NaN,NaN


#### **Data Cleaning and Pre-Proccessing**

Here, I want to keep HITId (represnting speeches), WorkerId, and Answer.manifesto_class.labels

In [116]:
# only keep needed columns and rename 
df_clean = df[['HITId', 'WorkerId', 'Answer.manifesto_class.labels']].copy() #remove unnecessary columns
df_clean.rename(columns={
    'HITId': 'speech_id',
    'WorkerId': 'worker_id',
    'Answer.manifesto_class.labels': 'answer'
}, inplace=True)

# extra cleaning, dropping nulls, making sure all labels are ints, 
df_clean = df_clean.sort_values(by=['speech_id', 'worker_id']).reset_index(drop=True)

# Final preview

df_clean

,speech_id,worker_id,answer
0,301KG0KXAZ355GRD9LSX6LBA2CHH2H,A1NBQ61Y6KO3O9,"[""Market regulation"",""Middle Class & Professio..."
1,301KG0KXAZ355GRD9LSX6LBA2CHH2H,A1X47COW2Y9SEL,"[""Market regulation""]"
2,301KG0KXAZ355GRD9LSX6LBA2CHH2H,A335R4YE2E34H6,"[""Market regulation""]"
3,302OLP89EMP3TBB1087B1CXS184CAA,A1NBQ61Y6KO3O9,"[""Government and administrative efficiency"",""E..."
4,302OLP89EMP3TBB1087B1CXS184CAA,A1X47COW2Y9SEL,"[""Economic Goals""]"
...,...,...,...
655,3ZXNP4Z3AE3EDD796IOQU4LPUM0L7M,A1X47COW2Y9SEL,"[""Market regulation"",""Government and administr..."
656,3ZXNP4Z3AE3EDD796IOQU4LPUM0L7M,A3GS2NTQ4XU059,"[""Free Market Economy""]"
657,3ZXV7Q5FKY6YP2S9BOT3FD8JYOHCFN,A1BQ37ZGUM16XI,"[""Market regulation""]"
658,3ZXV7Q5FKY6YP2S9BOT3FD8JYOHCFN,A1X47COW2Y9SEL,"[""Market regulation"",""Anti-Growth Economy""]"


define 'answer's and workers

In [117]:
workers = [
    'A1BQ37ZGUM16XI', 
    'A1NBQ61Y6KO3O9', 
    'A1X47COW2Y9SEL', 
    'A335R4YE2E34H6', 
    'A3GS2NTQ4XU059', 
    'APXGSUB250NVH'
]

In [ ]:
# only run this once!!!!!!!!!!!!!!!
# df_clean['answer'] = df_clean['answer'].apply(eval)  # Safely parse list from string

Create a secondary matrix with unique speech rows and worker columns

In [144]:
df_pivot = df_clean.pivot(index='speech_id', columns='worker_id', values='answer')
df_pivot = df_pivot.reindex(columns=workers)
df_pivot = df_pivot.fillna('[]')  # maybe change

df_pivot

worker_id,A1X47COW2Y9SEL,A1NBQ61Y6KO3O9,A3GS2NTQ4XU059,APXGSUB250NVH,A335R4YE2E34H6,A1BQ37ZGUM16XI
speech_id,,,,,,
301KG0KXAZ355GRD9LSX6LBA2CHH2H,[Market regulation],"[Market regulation, Middle Class & Professiona...",[],[],[Market regulation],[]
302OLP89EMP3TBB1087B1CXS184CAA,[Economic Goals],"[Government and administrative efficiency, Eco...",[],[],[],[]
306996CF7J2OPI6VQPINFRY7FTYB1E,[Government and administrative efficiency],"[Anti-Growth Economy, Market regulation]",[],[],"[Economic Planning, Economic Growth]",[]
3087LXLJ79YEVAXTZ80JKC6IPZKF0X,"[Free Market Economy, Economic Growth]","[Market regulation, Economic Planning, Economi...",[],[],"[Market regulation, Economic Growth]",[]
308KJXFUKEOKFSDCMU21QMXWPU4ATR,[Economic Goals],"[Economic Planning, Market regulation, Economi...",[],[],"[Market regulation, Economic Planning]",[]
...,...,...,...,...,...,...
3YZ7A3YHSSBMUMOG2Y6U86LOVAD5S5,"[Government and administrative efficiency, Fre...","[Government and administrative efficiency, Mar...",[Free Market Economy],[],[],[]
3ZUE82NE1XJWDZ2WUCDYDRC08AKF8Y,[],"[Corporatism / Mixed Economy, Anti-Growth Econ...",[],"[Economic Goals, Government and administrative...",[Corporatism / Mixed Economy],[]
3ZURAPD29V5E2ZIBVR4JYHPSURQF10,[Technology and Infrastructure],"[Middle Class & Professional Groups, Incentive...",[],[],"[Labor Groups Positive, Technology and Infrast...",[]


## Try 1 : Use a massive aggregate Dataframe (didn't work)

In [118]:

# turn all the answers into an array 
all_answers = df_clean['answer'].dropna().apply(ast.literal_eval)
all_categories = [category for sublist in all_answers for category in sublist]
unique_categories = pd.Series(all_categories).unique()

print(unique_categories) #check 


['Market regulation' 'Middle Class & Professional Groups'
 'Government and administrative efficiency' 'Economic Planning'
 'Economic Goals' 'Anti-Growth Economy' 'Economic Growth'
 'Free Market Economy' 'Incentives' 'Keynesian Demand Management'
 'Corporatism / Mixed Economy' 'Technology and Infrastructure'
 'Labor Groups Positive' 'Labor Groups Negative']


Now, we compute a 'count' matrix in order to analyze similary with fleiss kappa; here columns are (answer, annotator) pairs

In [119]:
# Clean and parse the 'answer' column into lists
df_clean['parsed_answer'] = df_clean['answer'].apply(
    lambda x: ast.literal_eval(x) if pd.notna(x) and x.strip() else []
)

# Get unique workers and categories
unique_workers = df_clean['worker_id'].unique()
all_categories = [cat for ans in df_clean['parsed_answer'] for cat in ans]
unique_categories = pd.Series(all_categories).unique()


print(unique_categories) #view


['Market regulation' 'Middle Class & Professional Groups'
 'Government and administrative efficiency' 'Economic Planning'
 'Economic Goals' 'Anti-Growth Economy' 'Economic Growth'
 'Free Market Economy' 'Incentives' 'Keynesian Demand Management'
 'Corporatism / Mixed Economy' 'Technology and Infrastructure'
 'Labor Groups Positive' 'Labor Groups Negative']


In [120]:
# construct binary matrix 
worker_category_pairs = list(product(unique_workers, unique_categories))
speech_ids = df_clean['speech_id'].unique()
binary_matrix = pd.DataFrame(0, index=speech_ids, columns=worker_category_pairs)

print(binary_matrix.columns) #view



Index([                       ('A1NBQ61Y6KO3O9', 'Market regulation'),
             ('A1NBQ61Y6KO3O9', 'Middle Class & Professional Groups'),
       ('A1NBQ61Y6KO3O9', 'Government and administrative efficiency'),
                              ('A1NBQ61Y6KO3O9', 'Economic Planning'),
                                 ('A1NBQ61Y6KO3O9', 'Economic Goals'),
                            ('A1NBQ61Y6KO3O9', 'Anti-Growth Economy'),
                                ('A1NBQ61Y6KO3O9', 'Economic Growth'),
                            ('A1NBQ61Y6KO3O9', 'Free Market Economy'),
                                     ('A1NBQ61Y6KO3O9', 'Incentives'),
                    ('A1NBQ61Y6KO3O9', 'Keynesian Demand Management'),
                    ('A1NBQ61Y6KO3O9', 'Corporatism / Mixed Economy'),
                  ('A1NBQ61Y6KO3O9', 'Technology and Infrastructure'),
                          ('A1NBQ61Y6KO3O9', 'Labor Groups Positive'),
                          ('A1NBQ61Y6KO3O9', 'Labor Groups Negative'),
      

In [121]:


# loop through speeches in binary_matrix - speech_id are the index and don't have a column
    # loop through workers - stored in workers[] array
        # if this worker mentioned its paired answer in this speech, assign this cell to 1
# Ensure columns are MultiIndex
if not isinstance(binary_matrix.columns, pd.MultiIndex):
    binary_matrix.columns = pd.MultiIndex.from_tuples(binary_matrix.columns)

# Get the list of speech IDs and workers
speech_ids = binary_matrix.index
workers = list(set(df_clean['worker_id']))

# Loop through each speech
for speech_id in speech_ids:
    # Filter df_clean to just this speech
    df_speech = df_clean[df_clean['speech_id'] == speech_id]

    # Loop through each worker
    for worker in workers:
        # Get the categories this worker annotated for this speech
        categories = df_speech[df_speech['worker_id'] == worker]['parsed_answer']
        if categories.empty:
            continue  # this worker didn't annotate this speech

        # categories is a Series of lists → flatten it
        mentioned_cats = set(cat for sublist in categories for cat in sublist)

        for category in mentioned_cats:
            key = (worker, category)
            if key in binary_matrix.columns:
                binary_matrix.loc[speech_id, key] = 1




binary_matrix #view 




A1NBQ61Y6KO3O9  \
                               Market regulation   
301KG0KXAZ355GRD9LSX6LBA2CHH2H                 1   
302OLP89EMP3TBB1087B1CXS184CAA                 0   
306996CF7J2OPI6VQPINFRY7FTYB1E                 1   
3087LXLJ79YEVAXTZ80JKC6IPZKF0X                 1   
308KJXFUKEOKFSDCMU21QMXWPU4ATR                 1   
...                                          ...   
3YZ7A3YHSSBMUMOG2Y6U86LOVAD5S5                 1   
3ZUE82NE1XJWDZ2WUCDYDRC08AKF8Y                 0   
3ZURAPD29V5E2ZIBVR4JYHPSURQF10                 0   
3ZXNP4Z3AE3EDD796IOQU4LPUM0L7M                 1   
3ZXV7Q5FKY6YP2S9BOT3FD8JYOHCFN                 0   

                                                                   \
                               Middle Class & Professional Groups   
301KG0KXAZ355GRD9LSX6LBA2CHH2H                                  1   
302OLP89EMP3TBB1087B1CXS184CAA                                  0   
306996CF7J2OPI6VQPINFRY7FTYB1E                                  0   
3087LXLJ79YEVAXTZ80JKC6IPZKF0X                                  0   
308KJXFUKEOKFSDCMU21QMXWPU4ATR                                  0   
...                                                           ...   
3YZ7A3YHSSBMUMOG2Y6U86LOVAD5S5                                  0   
3ZUE82NE1XJWDZ2WUCDYDRC08AKF8Y                                  0   
3ZURAPD29V5E2ZIBVR4JYHPSURQF10                                  1   
3ZXNP4Z3AE3EDD796IOQU4LPUM0L7M                                  1   
3ZXV7Q5FKY6YP2S9BOT3FD8JYOHCFN                                  0   

                                                                         \
                               Government and administrative efficiency   
301KG0KXAZ355GRD9LSX6LBA2CHH2H                                        0   
302OLP89EMP3TBB1087B1CXS184CAA                                        1   
306996CF7J2OPI6VQPINFRY7FTYB1E                                        0   
3087LXLJ79YEVAXTZ80JKC6IPZKF0X                                        0   
308KJXFUKEOKFSDCMU21QMXWPU4ATR                                        0   
...                                                                 ...   
3YZ7A3YHSSBMUMOG2Y6U86LOVAD5S5                                        1   
3ZUE82NE1XJWDZ2WUCDYDRC08AKF8Y                                        1   
3ZURAPD29V5E2ZIBVR4JYHPSURQF10                                        0   
3ZXNP4Z3AE3EDD796IOQU4LPUM0L7M                                        1   
3ZXV7Q5FKY6YP2S9BOT3FD8JYOHCFN                                        0   

                                                                 \
                               Economic Planning Economic Goals   
301KG0KXAZ355GRD9LSX6LBA2CHH2H                 0              0   
302OLP89EMP3TBB1087B1CXS184CAA                 1              1   
306996CF7J2OPI6VQPINFRY7FTYB1E                 0              0   
3087LXLJ79YEVAXTZ80JKC6IPZKF0X                 1              0   
308KJXFUKEOKFSDCMU21QMXWPU4ATR                 1              1   
...                                          ...            ...   
3YZ7A3YHSSBMUMOG2Y6U86LOVAD5S5                 0              1   
3ZUE82NE1XJWDZ2WUCDYDRC08AKF8Y                 0              0   
3ZURAPD29V5E2ZIBVR4JYHPSURQF10                 0              0   
3ZXNP4Z3AE3EDD796IOQU4LPUM0L7M                 0              0   
3ZXV7Q5FKY6YP2S9BOT3FD8JYOHCFN                 0              0   

                                                                    \
                               Anti-Growth Economy Economic Growth   
301KG0KXAZ355GRD9LSX6LBA2CHH2H                   0               0   
302OLP89EMP3TBB1087B1CXS184CAA                   0               0   
306996CF7J2OPI6VQPINFRY7FTYB1E                   1               0   
3087LXLJ79YEVAXTZ80JKC6IPZKF0X                   0               1   
308KJXFUKEOKFSDCMU21QMXWPU4ATR                   0               0   
...                                            ...             ...   
3YZ7A3YHSSBMUMOG2Y6U86LOVAD5S5                   0 

Check binary_matrix columns to ensure data aggregation worked 

In [122]:

binary_matrix

A1NBQ61Y6KO3O9  \
                               Market regulation   
301KG0KXAZ355GRD9LSX6LBA2CHH2H                 1   
302OLP89EMP3TBB1087B1CXS184CAA                 0   
306996CF7J2OPI6VQPINFRY7FTYB1E                 1   
3087LXLJ79YEVAXTZ80JKC6IPZKF0X                 1   
308KJXFUKEOKFSDCMU21QMXWPU4ATR                 1   
...                                          ...   
3YZ7A3YHSSBMUMOG2Y6U86LOVAD5S5                 1   
3ZUE82NE1XJWDZ2WUCDYDRC08AKF8Y                 0   
3ZURAPD29V5E2ZIBVR4JYHPSURQF10                 0   
3ZXNP4Z3AE3EDD796IOQU4LPUM0L7M                 1   
3ZXV7Q5FKY6YP2S9BOT3FD8JYOHCFN                 0   

                                                                   \
                               Middle Class & Professional Groups   
301KG0KXAZ355GRD9LSX6LBA2CHH2H                                  1   
302OLP89EMP3TBB1087B1CXS184CAA                                  0   
306996CF7J2OPI6VQPINFRY7FTYB1E                                  0   
3087LXLJ79YEVAXTZ80JKC6IPZKF0X                                  0   
308KJXFUKEOKFSDCMU21QMXWPU4ATR                                  0   
...                                                           ...   
3YZ7A3YHSSBMUMOG2Y6U86LOVAD5S5                                  0   
3ZUE82NE1XJWDZ2WUCDYDRC08AKF8Y                                  0   
3ZURAPD29V5E2ZIBVR4JYHPSURQF10                                  1   
3ZXNP4Z3AE3EDD796IOQU4LPUM0L7M                                  1   
3ZXV7Q5FKY6YP2S9BOT3FD8JYOHCFN                                  0   

                                                                         \
                               Government and administrative efficiency   
301KG0KXAZ355GRD9LSX6LBA2CHH2H                                        0   
302OLP89EMP3TBB1087B1CXS184CAA                                        1   
306996CF7J2OPI6VQPINFRY7FTYB1E                                        0   
3087LXLJ79YEVAXTZ80JKC6IPZKF0X                                        0   
308KJXFUKEOKFSDCMU21QMXWPU4ATR                                        0   
...                                                                 ...   
3YZ7A3YHSSBMUMOG2Y6U86LOVAD5S5                                        1   
3ZUE82NE1XJWDZ2WUCDYDRC08AKF8Y                                        1   
3ZURAPD29V5E2ZIBVR4JYHPSURQF10                                        0   
3ZXNP4Z3AE3EDD796IOQU4LPUM0L7M                                        1   
3ZXV7Q5FKY6YP2S9BOT3FD8JYOHCFN                                        0   

                                                                 \
                               Economic Planning Economic Goals   
301KG0KXAZ355GRD9LSX6LBA2CHH2H                 0              0   
302OLP89EMP3TBB1087B1CXS184CAA                 1              1   
306996CF7J2OPI6VQPINFRY7FTYB1E                 0              0   
3087LXLJ79YEVAXTZ80JKC6IPZKF0X                 1              0   
308KJXFUKEOKFSDCMU21QMXWPU4ATR                 1              1   
...                                          ...            ...   
3YZ7A3YHSSBMUMOG2Y6U86LOVAD5S5                 0              1   
3ZUE82NE1XJWDZ2WUCDYDRC08AKF8Y                 0              0   
3ZURAPD29V5E2ZIBVR4JYHPSURQF10                 0              0   
3ZXNP4Z3AE3EDD796IOQU4LPUM0L7M                 0              0   
3ZXV7Q5FKY6YP2S9BOT3FD8JYOHCFN                 0              0   

                                                                    \
                               Anti-Growth Economy Economic Growth   
301KG0KXAZ355GRD9LSX6LBA2CHH2H                   0               0   
302OLP89EMP3TBB1087B1CXS184CAA                   0               0   
306996CF7J2OPI6VQPINFRY7FTYB1E                   1               0   
3087LXLJ79YEVAXTZ80JKC6IPZKF0X                   0               1   
308KJXFUKEOKFSDCMU21QMXWPU4ATR                   0               0   
...                                            ...             ...   
3YZ7A3YHSSBMUMOG2Y6U86LOVAD5S5                   0 

In [123]:
row_sums = binary_matrix.sum(axis=1)
for i, total in enumerate(row_sums):
    print(f"Row {i} sum: {total}")

Row 0 sum: 4
Row 1 sum: 4
Row 2 sum: 5
Row 3 sum: 7
Row 4 sum: 6
Row 5 sum: 3
Row 6 sum: 4
Row 7 sum: 1
Row 8 sum: 3
Row 9 sum: 4
Row 10 sum: 8
Row 11 sum: 3
Row 12 sum: 4
Row 13 sum: 6
Row 14 sum: 9
Row 15 sum: 8
Row 16 sum: 0
Row 17 sum: 6
Row 18 sum: 11
Row 19 sum: 4
Row 20 sum: 2
Row 21 sum: 3
Row 22 sum: 4
Row 23 sum: 4
Row 24 sum: 3
Row 25 sum: 9
Row 26 sum: 2
Row 27 sum: 5
Row 28 sum: 2
Row 29 sum: 6
Row 30 sum: 10
Row 31 sum: 8
Row 32 sum: 5
Row 33 sum: 13
Row 34 sum: 7
Row 35 sum: 3
Row 36 sum: 3
Row 37 sum: 10
Row 38 sum: 6
Row 39 sum: 5
Row 40 sum: 0
Row 41 sum: 7
Row 42 sum: 11
Row 43 sum: 1
Row 44 sum: 4
Row 45 sum: 3
Row 46 sum: 5
Row 47 sum: 6
Row 48 sum: 8
Row 49 sum: 4
Row 50 sum: 5
Row 51 sum: 4
Row 52 sum: 1
Row 53 sum: 7
Row 54 sum: 3
Row 55 sum: 1
Row 56 sum: 5
Row 57 sum: 5
Row 58 sum: 4
Row 59 sum: 6
Row 60 sum: 2
Row 61 sum: 9
Row 62 sum: 4
Row 63 sum: 6
Row 64 sum: 8
Row 65 sum: 6
Row 66 sum: 4
Row 67 sum: 6
Row 68 sum: 1
Row 69 sum: 5
Row 70 sum: 5
Row 71 sum:

## Try 2: run Fleiss Kappa on each seperate category

## Try 3: run Krippendorff’s Alpha on annotations

 Krippendorff’s Alpha doesn't require that each annotator label once, so this measure will work for us

In [135]:
data = []
for _, row in df_clean.iterrows():
    speech = row['speech_id']
    worker = row['worker_id']
    for category in row['parsed_answer']:
        data.append((worker, speech, category))  # format: (coder, item, label)
data


[('A1NBQ61Y6KO3O9', '301KG0KXAZ355GRD9LSX6LBA2CHH2H', 'Market regulation'),
 ('A1NBQ61Y6KO3O9',
  '301KG0KXAZ355GRD9LSX6LBA2CHH2H',
  'Middle Class & Professional Groups'),
 ('A1X47COW2Y9SEL', '301KG0KXAZ355GRD9LSX6LBA2CHH2H', 'Market regulation'),
 ('A335R4YE2E34H6', '301KG0KXAZ355GRD9LSX6LBA2CHH2H', 'Market regulation'),
 ('A1NBQ61Y6KO3O9',
  '302OLP89EMP3TBB1087B1CXS184CAA',
  'Government and administrative efficiency'),
 ('A1NBQ61Y6KO3O9', '302OLP89EMP3TBB1087B1CXS184CAA', 'Economic Planning'),
 ('A1NBQ61Y6KO3O9', '302OLP89EMP3TBB1087B1CXS184CAA', 'Economic Goals'),
 ('A1X47COW2Y9SEL', '302OLP89EMP3TBB1087B1CXS184CAA', 'Economic Goals'),
 ('A1NBQ61Y6KO3O9', '306996CF7J2OPI6VQPINFRY7FTYB1E', 'Anti-Growth Economy'),
 ('A1NBQ61Y6KO3O9', '306996CF7J2OPI6VQPINFRY7FTYB1E', 'Market regulation'),
 ('A1X47COW2Y9SEL',
  '306996CF7J2OPI6VQPINFRY7FTYB1E',
  'Government and administrative efficiency'),
 ('A335R4YE2E34H6', '306996CF7J2OPI6VQPINFRY7FTYB1E', 'Economic Planning'),
 ('A335R4YE2E34H6

In [ ]:
task = agreement.AnnotationTask(data=data)

# print("Cohen's Kappa:", task.kappa())
# print("Fleiss' Kappa:", task.multi_kappa())
print("Krippendorff’s Alpha:", task.alpha())

Krippendorff’s Alpha: 0.06836551265048352


## Try 2: run pair wise Cohen Kappa